## 1. Installation (Colab)

https://github.com/Farama-Foundation/Gymnasium

https://gymnasium.farama.org/

https://gymnasium.farama.org/environments/toy_text/frozen_lake/

In [11]:
!pip install gymnasium==1.0.0

## 2. Entrainement

In [12]:
##########################
# 2. Import des packages
##########################
import gymnasium as gym
import numpy as np
import random

In [13]:
##########################
# 3. Création de l'environnement
##########################
is_slippery=False
env = gym.make('FrozenLake-v1', is_slippery=is_slippery)
# - is_slippery=True: le sol est glissant (difficile de contrôler les déplacements)
# - mode par défaut: 4x4

n_states = env.observation_space.n  # Nombre d'états (16 pour FrozenLake 4x4)
n_actions = env.action_space.n      # Nombre d'actions (4: gauche, bas, droite, haut)

In [14]:
##########################
# 4. Paramètres de Q-learning
##########################
alpha = 0.8       # Taux d'apprentissage
gamma = 0.95      # Facteur de discount
epsilon = 1.0     # Épsilon initial pour l'exploration (EPS-greedy)
epsilon_min = 0.01
epsilon_decay = 0.995

num_episodes = 10000

# Initialisation de la table Q
Q = np.zeros((n_states, n_actions))

In [15]:
##########################
# 5. Boucle d'apprentissage
##########################
all_rewards = []

for episode in range(num_episodes):
    state = env.reset()[0]  # reset() retourne un (observation, info) depuis Gym 0.26
    done = False
    total_reward = 0

    while not done:
        # Choix de l'action : exploration vs exploitation
        if np.random.rand() < epsilon:
            action = env.action_space.sample()  # exploration
        else:
            action = np.argmax(Q[state,:])       # exploitation

        # Effectuer l'action dans l'environnement
        next_state, reward, done, truncated, info = env.step(action)

        # Mettre à jour la valeur Q
        Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])

        # Passage à l'état suivant
        state = next_state
        total_reward += reward

    # Mise à jour de epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    all_rewards.append(total_reward)

## 3. Évaluation

In [16]:
##########################
# 6. Évaluation du résultat
##########################
# Taux de réussite moyen sur des paquets d'épisodes
window = 100
moving_avg = [np.mean(all_rewards[i:i+window]) for i in range(0, len(all_rewards), window)]
print(f"Q-Learning terminé. Moyenne de récompenses par tranche de {window} épisodes :\\n", moving_avg)

Q-Learning terminé. Moyenne de récompenses par tranche de 100 épisodes :\n [0.0, 0.01, 0.63, 0.84, 0.89, 0.9, 0.96, 1.0, 0.97, 0.98, 1.0, 0.99, 0.99, 1.0, 0.99, 0.98, 0.99, 0.99, 1.0, 0.99, 1.0, 1.0, 1.0, 0.99, 0.98, 0.99, 0.98, 1.0, 0.99, 1.0, 0.99, 1.0, 0.99, 1.0, 0.98, 0.99, 0.99, 0.99, 1.0, 0.99, 1.0, 0.99, 0.99, 0.98, 1.0, 0.99, 0.98, 1.0, 0.99, 0.99, 1.0, 0.99, 0.98, 0.98, 0.99, 0.99, 0.99, 1.0, 0.99, 1.0, 0.96, 0.98, 1.0, 0.98, 1.0, 1.0, 0.99, 0.99, 1.0, 0.99, 0.99, 0.99, 0.98, 0.98, 0.98, 0.98, 1.0, 0.99, 0.98, 0.98, 0.98, 0.98, 0.99, 0.99, 0.99, 1.0, 0.98, 1.0, 1.0, 0.97, 0.99, 0.97, 0.99, 0.99, 1.0, 0.98, 1.0, 1.0, 1.0, 1.0]


In [17]:
##########################
# 7. Test de la politique
##########################
def test_agent(env, Q, episodes=100):
    successes = 0
    steps = 0
    for _ in range(episodes):
        obs, info = env.reset()
        episode_over = False
        while not episode_over:
            action = np.argmax(Q[obs, :])  # meilleure action
            obs, reward, terminated, truncated, info = env.step(action)
            if terminated and reward == 1.0:
                successes += 1
            steps += 1

            episode_over = terminated or truncated
    return successes / episodes, steps / episodes

success_rate, steps = test_agent(env, Q, episodes=100)
print(f"Taux de réussite de l'agent sur 1000 épisodes de test : {success_rate*100:.2f}%")
print(f"Durée moyenne d'un épisode : {int(steps)} pas")

Taux de réussite de l'agent sur 1000 épisodes de test : 100.00%
Durée moyenne d'un épisode : 6 pas


## 4. Visualisation

### Politique optimale

In [18]:
import numpy as np

# Exemple de Q (table de q-valeurs) déjà entraînée : Q[state, action]
# Ici, Q est supposée être un numpy array de shape (16, 4) pour la grille 4x4
# Ex : Q = np.random.random((16,4))  # <- À remplacer par votre Q appris

# Symbole pour chaque action
actions_symbols = {
    0: '←',
    1: '↓',
    2: '→',
    3: '↑'
}

# Récupération de l'action la plus probable dans Q pour chaque état (0..15)
best_actions = np.argmax(Q, axis=1)

# Affichage de la grille 4x4
# Indices (états) :  0   1   2   3
#                    4   5   6   7
#                    8   9  10  11
#                   12  13  14  15
# Sur FrozenLake-v1, la config par défaut est :
# Row 0 : SFFF
# Row 1 : FHFH
# Row 2 : FFFH
# Row 3 : HFFG

env_map = [
    ['S','F','F','F'],
    ['F','H','F','H'],
    ['F','F','F','H'],
    ['H','F','F','G']
]

print("Politique optimale (d'après Q) :\n")

for i in range(4):
    row_str = ""
    for j in range(4):
        state = i*4 + j  # État courant
        cell_type = env_map[i][j]

        if cell_type in ['H', 'G', 'S']:
            # On affiche directement la case spéciale
            row_str += f" {cell_type} "
        else:
            # On récupère l'action optimale et on affiche son symbole
            action = best_actions[state]
            row_str += f" {actions_symbols[action]} "
    print(row_str)

Politique optimale (d'après Q) :

 S  ←  ←  ← 
 ↓  H  ↓  H 
 →  ↓  ↓  H 
 H  →  →  G 


### Vidéo

In [19]:
from IPython.display import HTML
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

In [20]:
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

frozen_lake = "FrozenLake-v1.mp4"

env = gym.make('FrozenLake-v1', is_slippery=is_slippery, render_mode="rgb_array")
env = RecordVideo(env, video_folder="frozen_lake", name_prefix="eval",
                  episode_trigger=lambda x: True)
env = RecordEpisodeStatistics(env)

obs, info = env.reset()

episode_over = False
while not episode_over:
    # action = env.action_space.sample()  # replace with actual agent
    action = np.argmax(Q[obs, :])
    obs, reward, terminated, truncated, info = env.step(action)

    episode_over = terminated or truncated

env.close()

print(f'Episode time taken: {env.time_queue}')
print(f'Episode total rewards: {env.return_queue}')
print(f'Episode lengths: {env.length_queue}')

html = render_mp4("frozen_lake/eval-episode-0.mp4")
HTML(html)

Episode time taken: deque([0.021965], maxlen=100)
Episode total rewards: deque([1.0], maxlen=100)
Episode lengths: deque([6], maxlen=100)


/usr/local/lib/python3.11/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/frozen_lake folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
